## Residual Network 구현 및 학습

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## Residual Unit 구현

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit,self).__init__()
        self.bn1 = tf.keras.layer.BatchNormalization()
        self.conv1 = tf.keras.layers.Con2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layer.BatchNormalization()
        self.conv2= tf.keras.layers.Con2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1),padding='same')
            
        def call(self, x ,training=False, mask=None):
            h = self.bn1(x, training=training)
            h = tf.nn.relu(h)
            h = self.conv1(h)
            
            h = self.bn2(x, training=training)
            h = tf.nn.relu(h)
            h = self.conv2(h)
            return self.identity(x) + h

## Residual Layer 구현

In [5]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = []
        for f_in, f_out in zip([filter_in] + list(filters),filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
            
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training = training)
        return x